# positivity와 Diagnosis code를 모두 고려

**Diagnosis code** 
+ 0~4 -> 0     
+ 5 -> 1

||positvity==1| positivity==0|
|---|---|---|
|Dignosis code == 1|63|14|
|Dignosis code == 0|37|54|


## Case 1
#### 신경심리검사 결과와 의사 진단 모두 알츠하이머인 경우 -> 2
#### 신경심리검사 결과와 의사 진단 둘 중 하나만 알츠하이머인 경우 -> 1  
#### 신경심리검사 결과와 의사 진단 모두 정상인 경우로 분류   -> 0

In [1]:
import pandas as pd

data = pd.read_csv('../알츠하이머/data/modeling.csv').iloc[:,1:]
data

,positivity,Sex,Age,Onset age,Eduction,Diagnosis code,APOE,SNSB,MMSE,CDR,...,GCP,PQ,MTC,CN,PUT,THA,Composite*,BS,Mid,Pons
0,0,1,70,70.0,7,0,0,1,30,0.0,...,1.181286,1.120224,1.212681,1.153254,1.290994,1.229675,1.125368,1.786,1.562976,1.884322
1,0,1,71,66.0,11,4,0,1,26,0.5,...,1.223034,1.156271,1.081563,1.174605,1.318370,1.221072,1.139724,1.794,1.500693,1.828633
2,0,1,70,69.0,16,3,0,1,25,0.5,...,1.239517,1.132322,1.177088,1.223519,1.338729,1.138704,1.166470,1.811,1.542578,2.005075
3,0,1,74,73.0,4,5,0,1,22,0.5,...,1.291466,1.170313,1.115316,1.143561,1.342879,1.260485,1.134957,1.840,1.517475,1.845699
4,0,1,78,75.0,12,0,0,1,30,0.0,...,1.152890,1.086830,1.060615,1.177075,1.236356,1.224865,1.054314,1.702,1.525412,1.619479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1,0,66,63.0,3,5,1,1,14,0.5,...,2.158729,2.056892,1.251478,1.614418,1.762425,1.656072,1.743311,1.716,1.556473,1.713264
164,1,0,67,65.0,6,5,1,1,20,1.0,...,1.973726,1.835785,1.328443,1.762946,1.809742,1.708223,1.740501,1.672,1.580051,1.628052
165,1,0,70,69.0,18,5,1,1,26,0.5,...,2.262631,2.078635,1.249556,1.751085,1.787068,1.521853,1.787363,1.784,1.566836,1.743362
166,1,0,64,64.0,11,5,1,1,18,0.5,...,1.909286,1.787781,1.144415,1.504897,1.673013,1.606195,1.604490,1.510,1.405606,1.458925


In [2]:
for i in range(168):
    pos = data.iloc[i,0]
    code = data.iloc[i,5]
    if code==5 and pos==1:
        data.loc[i, 'answer'] = 2
    elif code in [0,1,2,3,4] and pos==0:
        data.loc[i,'answer'] = 0
    else:
        data.loc[i,'answer'] = 1
        
data['answer'] = data['answer'].astype('int')
data

,positivity,Sex,Age,Onset age,Eduction,Diagnosis code,APOE,SNSB,MMSE,CDR,...,PQ,MTC,CN,PUT,THA,Composite*,BS,Mid,Pons,answer
0,0,1,70,70.0,7,0,0,1,30,0.0,...,1.120224,1.212681,1.153254,1.290994,1.229675,1.125368,1.786,1.562976,1.884322,0
1,0,1,71,66.0,11,4,0,1,26,0.5,...,1.156271,1.081563,1.174605,1.318370,1.221072,1.139724,1.794,1.500693,1.828633,0
2,0,1,70,69.0,16,3,0,1,25,0.5,...,1.132322,1.177088,1.223519,1.338729,1.138704,1.166470,1.811,1.542578,2.005075,0
3,0,1,74,73.0,4,5,0,1,22,0.5,...,1.170313,1.115316,1.143561,1.342879,1.260485,1.134957,1.840,1.517475,1.845699,1
4,0,1,78,75.0,12,0,0,1,30,0.0,...,1.086830,1.060615,1.177075,1.236356,1.224865,1.054314,1.702,1.525412,1.619479,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1,0,66,63.0,3,5,1,1,14,0.5,...,2.056892,1.251478,1.614418,1.762425,1.656072,1.743311,1.716,1.556473,1.713264,2
164,1,0,67,65.0,6,5,1,1,20,1.0,...,1.835785,1.328443,1.762946,1.809742,1.708223,1.740501,1.672,1.580051,1.628052,2
165,1,0,70,69.0,18,5,1,1,26,0.5,...,2.078635,1.249556,1.751085,1.787068,1.521853,1.787363,1.784,1.566836,1.743362,2
166,1,0,64,64.0,11,5,1,1,18,0.5,...,1.787781,1.144415,1.504897,1.673013,1.606195,1.604490,1.510,1.405606,1.458925,2


In [3]:
data = data.drop(['positivity','Diagnosis code'], axis=1)
data

,Sex,Age,Onset age,Eduction,APOE,SNSB,MMSE,CDR,CDR-SOB,GDS,...,PQ,MTC,CN,PUT,THA,Composite*,BS,Mid,Pons,answer
0,1,70,70.0,7,0,1,30,0.0,0.0,1,...,1.120224,1.212681,1.153254,1.290994,1.229675,1.125368,1.786,1.562976,1.884322,0
1,1,71,66.0,11,0,1,26,0.5,1.5,3,...,1.156271,1.081563,1.174605,1.318370,1.221072,1.139724,1.794,1.500693,1.828633,0
2,1,70,69.0,16,0,1,25,0.5,1.0,3,...,1.132322,1.177088,1.223519,1.338729,1.138704,1.166470,1.811,1.542578,2.005075,0
3,1,74,73.0,4,0,1,22,0.5,4.0,4,...,1.170313,1.115316,1.143561,1.342879,1.260485,1.134957,1.840,1.517475,1.845699,1
4,1,78,75.0,12,0,1,30,0.0,0.0,1,...,1.086830,1.060615,1.177075,1.236356,1.224865,1.054314,1.702,1.525412,1.619479,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,0,66,63.0,3,1,1,14,0.5,3.0,4,...,2.056892,1.251478,1.614418,1.762425,1.656072,1.743311,1.716,1.556473,1.713264,2
164,0,67,65.0,6,1,1,20,1.0,7.0,5,...,1.835785,1.328443,1.762946,1.809742,1.708223,1.740501,1.672,1.580051,1.628052,2
165,0,70,69.0,18,1,1,26,0.5,3.0,4,...,2.078635,1.249556,1.751085,1.787068,1.521853,1.787363,1.784,1.566836,1.743362,2
166,0,64,64.0,11,1,1,18,0.5,3.0,4,...,1.787781,1.144415,1.504897,1.673013,1.606195,1.604490,1.510,1.405606,1.458925,2


In [4]:
put = data.drop(['answer'], axis=1)
target = data['answer']

In [22]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(put, target, test_size=0.2, stratify= target, random_state=42)

In [23]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(train_x, train_y)

RandomForestClassifier(n_estimators=200)

In [24]:
pred_rfc = rfc.predict(test_x)

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

print(accuracy_score(test_y,pred_rfc))
print(f1_score(test_y,pred_rfc,average='macro'))
print(recall_score(test_y,pred_rfc,average='macro'))
print(precision_score(test_y,pred_rfc,average='macro'))
print(confusion_matrix(test_y,pred_rfc))

0.8823529411764706
0.8838383838383838
0.8897435897435897
0.8863636363636364
[[11  0  0]
 [ 0  9  1]
 [ 0  3 10]]


In [25]:
from xgboost.sklearn import XGBClassifier

xgbc = XGBClassifier(n_estimators=150)
xgbc.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=150, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [26]:
pred_xgbc = xgbc.predict(test_x)

print(accuracy_score(test_y,pred_xgbc))
print(f1_score(test_y,pred_xgbc,average='macro'))
print(recall_score(test_y,pred_xgbc,average='macro'))
print(precision_score(test_y,pred_xgbc,average='macro'))
print(confusion_matrix(test_y,pred_xgbc))

0.8235294117647058
0.8188405797101449
0.8230769230769232
0.8166666666666668
[[11  0  0]
 [ 1  7  2]
 [ 0  3 10]]


In [27]:
corr = data.corr()

## 다소 강한 상관관계 

In [28]:
lt = corr[(corr.answer>=0.4)|(corr.answer<=-0.4)].index
corr.iloc[-1].sort_values(ascending=False)

answer        1.000000
PQ            0.768317
GCP           0.767792
GCA           0.765438
PC            0.748501
Composite*    0.746448
LTC           0.740053
FC            0.739949
PUT           0.720270
CN            0.705349
THA           0.689530
OC            0.664857
GDS           0.650959
CDR-SOB       0.596691
APOE          0.411204
CDR           0.367450
MTC           0.199919
Age           0.186690
Onset age     0.148340
Eduction      0.147539
SGDepS       -0.015219
SNSB         -0.087940
Sex          -0.136458
BS           -0.198852
Mid          -0.295166
Pons         -0.316207
MMSE         -0.582262
Name: answer, dtype: float64

In [29]:
lt

Index(['APOE', 'MMSE', 'CDR-SOB', 'GDS', 'FC', 'LTC', 'PC', 'OC', 'GCA', 'GCP',
       'PQ', 'CN', 'PUT', 'THA', 'Composite*', 'answer'],
      dtype='object')

In [30]:
df = data[lt]
put = df.drop(['answer'], axis=1)
target = df['answer']

In [31]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(put, target, test_size=0.2, stratify= target, random_state=42)

## RF

In [32]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=150)
rfc.fit(train_x, train_y)

RandomForestClassifier(n_estimators=150)

In [33]:
pred_rfc = rfc.predict(test_x)

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

print(accuracy_score(test_y,pred_rfc))
print(f1_score(test_y,pred_rfc,average='macro'))
print(recall_score(test_y,pred_rfc,average='macro'))
print(precision_score(test_y,pred_rfc,average='macro'))
print(confusion_matrix(test_y,pred_rfc))

0.9117647058823529
0.9103313840155945
0.9076923076923077
0.9153439153439153
[[11  0  0]
 [ 0  8  2]
 [ 0  1 12]]


## XGB

In [34]:
from xgboost.sklearn import XGBClassifier

xgbc = XGBClassifier(n_estimators=100)
xgbc.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [35]:
pred_xgbc = xgbc.predict(test_x)

print(accuracy_score(test_y,pred_xgbc))
print(f1_score(test_y,pred_xgbc,average='macro'))
print(recall_score(test_y,pred_xgbc,average='macro'))
print(precision_score(test_y,pred_xgbc,average='macro'))
print(confusion_matrix(test_y,pred_xgbc))

0.8235294117647058
0.8188405797101449
0.8230769230769232
0.8166666666666668
[[11  0  0]
 [ 1  7  2]
 [ 0  3 10]]


## 강한 상관관계

In [36]:
lt = corr[(corr.answer>=0.7)|(corr.answer<=-0.7)].index
corr.iloc[-1].sort_values(ascending=False)

answer        1.000000
PQ            0.768317
GCP           0.767792
GCA           0.765438
PC            0.748501
Composite*    0.746448
LTC           0.740053
FC            0.739949
PUT           0.720270
CN            0.705349
THA           0.689530
OC            0.664857
GDS           0.650959
CDR-SOB       0.596691
APOE          0.411204
CDR           0.367450
MTC           0.199919
Age           0.186690
Onset age     0.148340
Eduction      0.147539
SGDepS       -0.015219
SNSB         -0.087940
Sex          -0.136458
BS           -0.198852
Mid          -0.295166
Pons         -0.316207
MMSE         -0.582262
Name: answer, dtype: float64

In [37]:
df = data[lt]
put = df.drop(['answer'], axis=1)
target = df['answer']

In [38]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(put, target, test_size=0.2, stratify= target, random_state=42)

In [39]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=150)
rfc.fit(train_x, train_y)

RandomForestClassifier(n_estimators=150)

In [40]:
pred_rfc = rfc.predict(test_x)

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

print(accuracy_score(test_y,pred_rfc))
print(f1_score(test_y,pred_rfc,average='macro'))
print(recall_score(test_y,pred_rfc,average='macro'))
print(precision_score(test_y,pred_rfc,average='macro'))
print(confusion_matrix(test_y,pred_rfc))

0.6470588235294118
0.5970764617691154
0.6214452214452214
0.5902777777777778
[[ 9  2  0]
 [ 3  2  5]
 [ 0  2 11]]


In [41]:
from xgboost.sklearn import XGBClassifier

xgbc = XGBClassifier(n_estimators=150)
xgbc.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=150, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [42]:
pred_xgbc = xgbc.predict(test_x)

print(accuracy_score(test_y,pred_xgbc))
print(f1_score(test_y,pred_xgbc,average='macro'))
print(recall_score(test_y,pred_xgbc,average='macro'))
print(precision_score(test_y,pred_xgbc,average='macro'))
print(confusion_matrix(test_y,pred_xgbc))

0.7352941176470589
0.6898550724637681
0.7107226107226108
0.7130718954248366
[[10  1  0]
 [ 2  3  5]
 [ 0  1 12]]


## Case 2
#### 신경심리검사 결과와 의사 진단 모두 알츠하이머인 경우 ->  1
#### 신경심리검사 결과와 의사 진단 둘 중 하나라도 알츠하이머가 아닌 경우 ->  0  

In [13]:
import pandas as pd

data = pd.read_csv('../알츠하이머/data/modeling.csv').iloc[:,1:]
data

,positivity,Sex,Age,Onset age,Eduction,Diagnosis code,APOE,SNSB,MMSE,CDR,...,GCP,PQ,MTC,CN,PUT,THA,Composite*,BS,Mid,Pons
0,0,1,70,70.0,7,0,0,1,30,0.0,...,1.181286,1.120224,1.212681,1.153254,1.290994,1.229675,1.125368,1.786,1.562976,1.884322
1,0,1,71,66.0,11,4,0,1,26,0.5,...,1.223034,1.156271,1.081563,1.174605,1.318370,1.221072,1.139724,1.794,1.500693,1.828633
2,0,1,70,69.0,16,3,0,1,25,0.5,...,1.239517,1.132322,1.177088,1.223519,1.338729,1.138704,1.166470,1.811,1.542578,2.005075
3,0,1,74,73.0,4,5,0,1,22,0.5,...,1.291466,1.170313,1.115316,1.143561,1.342879,1.260485,1.134957,1.840,1.517475,1.845699
4,0,1,78,75.0,12,0,0,1,30,0.0,...,1.152890,1.086830,1.060615,1.177075,1.236356,1.224865,1.054314,1.702,1.525412,1.619479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1,0,66,63.0,3,5,1,1,14,0.5,...,2.158729,2.056892,1.251478,1.614418,1.762425,1.656072,1.743311,1.716,1.556473,1.713264
164,1,0,67,65.0,6,5,1,1,20,1.0,...,1.973726,1.835785,1.328443,1.762946,1.809742,1.708223,1.740501,1.672,1.580051,1.628052
165,1,0,70,69.0,18,5,1,1,26,0.5,...,2.262631,2.078635,1.249556,1.751085,1.787068,1.521853,1.787363,1.784,1.566836,1.743362
166,1,0,64,64.0,11,5,1,1,18,0.5,...,1.909286,1.787781,1.144415,1.504897,1.673013,1.606195,1.604490,1.510,1.405606,1.458925


In [14]:
for i in range(168):
    pos = data.iloc[i,0]
    code = data.iloc[i,5]
    if code==5 and pos==1:
        data.loc[i, 'answer'] = 1
    else:
        data.loc[i,'answer'] = 0
        
data['answer'] = data['answer'].astype('int')
data

,positivity,Sex,Age,Onset age,Eduction,Diagnosis code,APOE,SNSB,MMSE,CDR,...,PQ,MTC,CN,PUT,THA,Composite*,BS,Mid,Pons,answer
0,0,1,70,70.0,7,0,0,1,30,0.0,...,1.120224,1.212681,1.153254,1.290994,1.229675,1.125368,1.786,1.562976,1.884322,0
1,0,1,71,66.0,11,4,0,1,26,0.5,...,1.156271,1.081563,1.174605,1.318370,1.221072,1.139724,1.794,1.500693,1.828633,0
2,0,1,70,69.0,16,3,0,1,25,0.5,...,1.132322,1.177088,1.223519,1.338729,1.138704,1.166470,1.811,1.542578,2.005075,0
3,0,1,74,73.0,4,5,0,1,22,0.5,...,1.170313,1.115316,1.143561,1.342879,1.260485,1.134957,1.840,1.517475,1.845699,0
4,0,1,78,75.0,12,0,0,1,30,0.0,...,1.086830,1.060615,1.177075,1.236356,1.224865,1.054314,1.702,1.525412,1.619479,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1,0,66,63.0,3,5,1,1,14,0.5,...,2.056892,1.251478,1.614418,1.762425,1.656072,1.743311,1.716,1.556473,1.713264,1
164,1,0,67,65.0,6,5,1,1,20,1.0,...,1.835785,1.328443,1.762946,1.809742,1.708223,1.740501,1.672,1.580051,1.628052,1
165,1,0,70,69.0,18,5,1,1,26,0.5,...,2.078635,1.249556,1.751085,1.787068,1.521853,1.787363,1.784,1.566836,1.743362,1
166,1,0,64,64.0,11,5,1,1,18,0.5,...,1.787781,1.144415,1.504897,1.673013,1.606195,1.604490,1.510,1.405606,1.458925,1


In [16]:
data = data.drop(['positivity','Diagnosis code'], axis=1)
data

,Sex,Age,Onset age,Eduction,APOE,SNSB,MMSE,CDR,CDR-SOB,GDS,...,PQ,MTC,CN,PUT,THA,Composite*,BS,Mid,Pons,answer
0,1,70,70.0,7,0,1,30,0.0,0.0,1,...,1.120224,1.212681,1.153254,1.290994,1.229675,1.125368,1.786,1.562976,1.884322,0
1,1,71,66.0,11,0,1,26,0.5,1.5,3,...,1.156271,1.081563,1.174605,1.318370,1.221072,1.139724,1.794,1.500693,1.828633,0
2,1,70,69.0,16,0,1,25,0.5,1.0,3,...,1.132322,1.177088,1.223519,1.338729,1.138704,1.166470,1.811,1.542578,2.005075,0
3,1,74,73.0,4,0,1,22,0.5,4.0,4,...,1.170313,1.115316,1.143561,1.342879,1.260485,1.134957,1.840,1.517475,1.845699,0
4,1,78,75.0,12,0,1,30,0.0,0.0,1,...,1.086830,1.060615,1.177075,1.236356,1.224865,1.054314,1.702,1.525412,1.619479,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,0,66,63.0,3,1,1,14,0.5,3.0,4,...,2.056892,1.251478,1.614418,1.762425,1.656072,1.743311,1.716,1.556473,1.713264,1
164,0,67,65.0,6,1,1,20,1.0,7.0,5,...,1.835785,1.328443,1.762946,1.809742,1.708223,1.740501,1.672,1.580051,1.628052,1
165,0,70,69.0,18,1,1,26,0.5,3.0,4,...,2.078635,1.249556,1.751085,1.787068,1.521853,1.787363,1.784,1.566836,1.743362,1
166,0,64,64.0,11,1,1,18,0.5,3.0,4,...,1.787781,1.144415,1.504897,1.673013,1.606195,1.604490,1.510,1.405606,1.458925,1


In [46]:
put = data.drop(['answer'], axis=1)
target = data['answer']

In [47]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(put, target, test_size=0.25, stratify= target, random_state=42)

In [48]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(train_x, train_y)

RandomForestClassifier()

In [49]:
pred_rfc = rfc.predict(test_x)
proba_rfc = rfc.predict_proba(test_x)[:,1]

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score

print(accuracy_score(test_y,pred_rfc))
print(f1_score(test_y,pred_rfc,average='macro'))
print(recall_score(test_y,pred_rfc,average='macro'))

print(precision_score(test_y,pred_rfc,average='macro'))
# print(roc_auc_score(test_y, proba_rfc))
print(confusion_matrix(test_y,pred_rfc))

0.9047619047619048
0.9027777777777778
0.9230769230769231
0.9
[[22  4]
 [ 0 16]]


In [50]:
xgbc = XGBClassifier()
xgbc.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [51]:
pred_xgbc = xgbc.predict(test_x)

print(accuracy_score(test_y,pred_xgbc))
print(f1_score(test_y,pred_xgbc,average='macro'))
print(recall_score(test_y,pred_xgbc,average='macro'))
print(precision_score(test_y,pred_xgbc,average='macro'))
print(confusion_matrix(test_y,pred_xgbc))

0.9285714285714286
0.926530612244898
0.9423076923076923
0.9210526315789473
[[23  3]
 [ 0 16]]


In [52]:
corr = data.corr()
lt = corr[(corr.answer>=0.4)|(corr.answer<=-0.4)].index
corr.iloc[-1].sort_values(ascending=False)

answer        1.000000
PQ            0.642173
GCA           0.636639
GCP           0.636622
PC            0.623585
LTC           0.619473
Composite*    0.617046
PUT           0.607060
FC            0.603318
THA           0.588992
CN            0.586593
GDS           0.586094
CDR-SOB       0.582376
OC            0.550451
APOE          0.402997
CDR           0.357398
MTC           0.221848
Eduction      0.191153
Age           0.089440
Onset age     0.056545
SGDepS       -0.070927
SNSB         -0.099900
Sex          -0.104328
BS           -0.203144
Mid          -0.244004
Pons         -0.315194
MMSE         -0.512052
Name: answer, dtype: float64

In [53]:
df = data[lt]
put = df.drop(['answer'], axis=1)
target = df['answer']

In [54]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(put, target, test_size=0.25, stratify= target, random_state=0)

## RF

In [55]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(train_x, train_y)

RandomForestClassifier()

In [56]:
pred_rfc = rfc.predict(test_x)
proba_rfc = rfc.predict_proba(test_x)[:,1]

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score

print(accuracy_score(test_y,pred_rfc))
print(f1_score(test_y,pred_rfc,average='macro'))
print(recall_score(test_y,pred_rfc,average='macro'))
print(precision_score(test_y,pred_rfc,average='macro'))
print(precision_score(test_y,pred_rfc,average='macro'))
# print(roc_auc_score(test_y, proba_rfc))
print(confusion_matrix(test_y,pred_rfc))

0.8809523809523809
0.8721850273889227
0.8677884615384616
0.8777777777777778
0.8777777777777778
[[24  2]
 [ 3 13]]


## XGB

In [57]:
xgbc = XGBClassifier()
xgbc.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [58]:
pred_xgbc = xgbc.predict(test_x)

print(accuracy_score(test_y,pred_xgbc))
print(f1_score(test_y,pred_xgbc,average='macro'))
print(recall_score(test_y,pred_xgbc,average='macro'))
print(precision_score(test_y,pred_xgbc,average='macro'))
print(confusion_matrix(test_y,pred_xgbc))

0.9285714285714286
0.9233110164333536
0.9182692307692308
0.9296296296296296
[[25  1]
 [ 2 14]]


# Case 3
    
#### Diagnosis code 3 ~ 5 and positivity 0 -> 1
#### Diagnosis code  5 and positivity 1 -> 1
#### else 0

In [19]:
import pandas as pd

data = pd.read_csv('../알츠하이머/data/modeling.csv').iloc[:,1:]
data

,positivity,Sex,Age,Onset age,Eduction,Diagnosis code,APOE,SNSB,MMSE,CDR,...,GCP,PQ,MTC,CN,PUT,THA,Composite*,BS,Mid,Pons
0,0,1,70,70.0,7,0,0,1,30,0.0,...,1.181286,1.120224,1.212681,1.153254,1.290994,1.229675,1.125368,1.786,1.562976,1.884322
1,0,1,71,66.0,11,4,0,1,26,0.5,...,1.223034,1.156271,1.081563,1.174605,1.318370,1.221072,1.139724,1.794,1.500693,1.828633
2,0,1,70,69.0,16,3,0,1,25,0.5,...,1.239517,1.132322,1.177088,1.223519,1.338729,1.138704,1.166470,1.811,1.542578,2.005075
3,0,1,74,73.0,4,5,0,1,22,0.5,...,1.291466,1.170313,1.115316,1.143561,1.342879,1.260485,1.134957,1.840,1.517475,1.845699
4,0,1,78,75.0,12,0,0,1,30,0.0,...,1.152890,1.086830,1.060615,1.177075,1.236356,1.224865,1.054314,1.702,1.525412,1.619479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1,0,66,63.0,3,5,1,1,14,0.5,...,2.158729,2.056892,1.251478,1.614418,1.762425,1.656072,1.743311,1.716,1.556473,1.713264
164,1,0,67,65.0,6,5,1,1,20,1.0,...,1.973726,1.835785,1.328443,1.762946,1.809742,1.708223,1.740501,1.672,1.580051,1.628052
165,1,0,70,69.0,18,5,1,1,26,0.5,...,2.262631,2.078635,1.249556,1.751085,1.787068,1.521853,1.787363,1.784,1.566836,1.743362
166,1,0,64,64.0,11,5,1,1,18,0.5,...,1.909286,1.787781,1.144415,1.504897,1.673013,1.606195,1.604490,1.510,1.405606,1.458925


In [20]:
for i in range(168):
    pos = data.iloc[i,0]
    code = data.iloc[i,5]
    if code in [3,4,5] and pos==1:
        data.loc[i, 'answer'] = 1
    elif code==5 and pos==0:
        data.loc[i, 'answer'] = 1
    else:
        data.loc[i,'answer'] = 0
        
data['answer'] = data['answer'].astype('int')
data

,positivity,Sex,Age,Onset age,Eduction,Diagnosis code,APOE,SNSB,MMSE,CDR,...,PQ,MTC,CN,PUT,THA,Composite*,BS,Mid,Pons,answer
0,0,1,70,70.0,7,0,0,1,30,0.0,...,1.120224,1.212681,1.153254,1.290994,1.229675,1.125368,1.786,1.562976,1.884322,0
1,0,1,71,66.0,11,4,0,1,26,0.5,...,1.156271,1.081563,1.174605,1.318370,1.221072,1.139724,1.794,1.500693,1.828633,0
2,0,1,70,69.0,16,3,0,1,25,0.5,...,1.132322,1.177088,1.223519,1.338729,1.138704,1.166470,1.811,1.542578,2.005075,0
3,0,1,74,73.0,4,5,0,1,22,0.5,...,1.170313,1.115316,1.143561,1.342879,1.260485,1.134957,1.840,1.517475,1.845699,1
4,0,1,78,75.0,12,0,0,1,30,0.0,...,1.086830,1.060615,1.177075,1.236356,1.224865,1.054314,1.702,1.525412,1.619479,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1,0,66,63.0,3,5,1,1,14,0.5,...,2.056892,1.251478,1.614418,1.762425,1.656072,1.743311,1.716,1.556473,1.713264,1
164,1,0,67,65.0,6,5,1,1,20,1.0,...,1.835785,1.328443,1.762946,1.809742,1.708223,1.740501,1.672,1.580051,1.628052,1
165,1,0,70,69.0,18,5,1,1,26,0.5,...,2.078635,1.249556,1.751085,1.787068,1.521853,1.787363,1.784,1.566836,1.743362,1
166,1,0,64,64.0,11,5,1,1,18,0.5,...,1.787781,1.144415,1.504897,1.673013,1.606195,1.604490,1.510,1.405606,1.458925,1


In [21]:
data = data.drop(['positivity','Diagnosis code'], axis=1)
data

,Sex,Age,Onset age,Eduction,APOE,SNSB,MMSE,CDR,CDR-SOB,GDS,...,PQ,MTC,CN,PUT,THA,Composite*,BS,Mid,Pons,answer
0,1,70,70.0,7,0,1,30,0.0,0.0,1,...,1.120224,1.212681,1.153254,1.290994,1.229675,1.125368,1.786,1.562976,1.884322,0
1,1,71,66.0,11,0,1,26,0.5,1.5,3,...,1.156271,1.081563,1.174605,1.318370,1.221072,1.139724,1.794,1.500693,1.828633,0
2,1,70,69.0,16,0,1,25,0.5,1.0,3,...,1.132322,1.177088,1.223519,1.338729,1.138704,1.166470,1.811,1.542578,2.005075,0
3,1,74,73.0,4,0,1,22,0.5,4.0,4,...,1.170313,1.115316,1.143561,1.342879,1.260485,1.134957,1.840,1.517475,1.845699,1
4,1,78,75.0,12,0,1,30,0.0,0.0,1,...,1.086830,1.060615,1.177075,1.236356,1.224865,1.054314,1.702,1.525412,1.619479,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,0,66,63.0,3,1,1,14,0.5,3.0,4,...,2.056892,1.251478,1.614418,1.762425,1.656072,1.743311,1.716,1.556473,1.713264,1
164,0,67,65.0,6,1,1,20,1.0,7.0,5,...,1.835785,1.328443,1.762946,1.809742,1.708223,1.740501,1.672,1.580051,1.628052,1
165,0,70,69.0,18,1,1,26,0.5,3.0,4,...,2.078635,1.249556,1.751085,1.787068,1.521853,1.787363,1.784,1.566836,1.743362,1
166,0,64,64.0,11,1,1,18,0.5,3.0,4,...,1.787781,1.144415,1.504897,1.673013,1.606195,1.604490,1.510,1.405606,1.458925,1


In [22]:
len(data[data.answer==1])

108

In [23]:
corr = data.corr()
lt = corr[(corr.answer<=-0.4)|(corr.answer>=0.4)].index
lt

Index(['MMSE', 'CDR-SOB', 'GDS', 'FC', 'LTC', 'PC', 'OC', 'GCA', 'GCP', 'PQ',
       'CN', 'PUT', 'THA', 'Composite*', 'answer'],
      dtype='object')

In [5]:
put = data.drop(['answer'], axis=1)
target = data['answer']

In [6]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(put, target, test_size=0.2, stratify= target, random_state=42)

In [7]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(train_x, train_y)

RandomForestClassifier()

In [8]:
pred_rfc = rfc.predict(test_x)
proba_rfc = rfc.predict_proba(test_x)[:,1]

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score

print(accuracy_score(test_y,pred_rfc))
print(f1_score(test_y,pred_rfc,average='macro'))
print(recall_score(test_y,pred_rfc,average='macro'))
print(precision_score(test_y,pred_rfc,average='macro'))
print(confusion_matrix(test_y,pred_rfc))

0.9705882352941176
0.9671497584541062
0.9583333333333333
0.9782608695652174
[[11  1]
 [ 0 22]]


In [9]:
from xgboost.sklearn import XGBClassifier

xgbc = XGBClassifier(n_estimators=150)
xgbc.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=150, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [10]:
pred_xgbc = xgbc.predict(test_x)

print(accuracy_score(test_y,pred_xgbc))
print(f1_score(test_y,pred_xgbc,average='macro'))
print(recall_score(test_y,pred_xgbc,average='macro'))
print(precision_score(test_y,pred_xgbc,average='macro'))
print(confusion_matrix(test_y,pred_xgbc))

0.9705882352941176
0.9683720930232559
0.9772727272727273
0.9615384615384616
[[12  0]
 [ 1 21]]


In [11]:
corr = data.corr()
lt = corr[(corr.answer>=0.4)|(corr.answer<=-0.4)].index
corr.iloc[-1].sort_values(ascending=False)

answer        1.000000
GCP           0.664150
GCA           0.658137
PQ            0.653714
FC            0.652239
Composite*    0.641270
GDS           0.639166
PC            0.631852
LTC           0.629375
PUT           0.613983
CN            0.600768
THA           0.569481
OC            0.553114
CDR-SOB       0.499855
APOE          0.319246
CDR           0.299344
Age           0.179990
Onset age     0.151019
MTC           0.098847
Eduction      0.097784
SGDepS        0.063815
SNSB         -0.057677
Sex          -0.131762
BS           -0.161264
Pons         -0.240374
Mid          -0.268018
MMSE         -0.551808
Name: answer, dtype: float64

In [12]:
df = data[lt]
put = df.drop(['answer'], axis=1)
target = df['answer']

In [13]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(put, target, test_size=0.2, stratify= target, random_state=42)

In [14]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(train_x, train_y)

RandomForestClassifier()

In [15]:
pred_rfc = rfc.predict(test_x)
proba_rfc = rfc.predict_proba(test_x)[:,1]

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score

print(accuracy_score(test_y,pred_rfc))
print(f1_score(test_y,pred_rfc,average='macro'))
print(recall_score(test_y,pred_rfc,average='macro'))
print(precision_score(test_y,pred_rfc,average='macro'))
print(precision_score(test_y,pred_rfc,average='macro'))
# print(roc_auc_score(test_y, proba_rfc))
print(confusion_matrix(test_y,pred_rfc))

1.0
1.0
1.0
1.0
1.0
[[12  0]
 [ 0 22]]


In [16]:
from xgboost.sklearn import XGBClassifier

xgbc = XGBClassifier(n_estimators=150)
xgbc.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=150, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [17]:
pred_xgbc = xgbc.predict(test_x)

print(accuracy_score(test_y,pred_xgbc))
print(f1_score(test_y,pred_xgbc,average='macro'))
print(recall_score(test_y,pred_xgbc,average='macro'))
print(precision_score(test_y,pred_xgbc,average='macro'))
print(confusion_matrix(test_y,pred_xgbc))

1.0
1.0
1.0
1.0
[[12  0]
 [ 0 22]]
